In [29]:
import os.path
from googleapiclient.discovery import build
from google.oauth2 import service_account



SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
BASE_DIR = os.path.dirname(os.path.abspath("scripts.py"))
SERVICE_ACCOUNT_FILE = os.path.join(BASE_DIR, 'local-talent-364913-14cc27db9b15.json')

credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# The ID and range of a sample spreadsheet.

SAMPLE_SPREADSHEET_ID = '1oYSVwlNr2NZSB6i2pNHyKN0aW34Y50jldQgEKHUJiVw'
SAMPLE_RANGE_NAME = 'Real_Data'


service = build('sheets', 'v4', credentials=credentials)

sheet = service.spreadsheets()
results = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
reasons = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range="Reasons").execute()

In [19]:
result = results["values"][2:]

In [21]:
j = ''
for i in result:
    if i[0] == '':
        i[0] = j
    j = i [0]
result

[['Гриб Анастасия',
  'Предложи эксперта для мероприятия в школе и согласование',
  ' -',
  ' -',
  ' -',
  ' -'],
 ['Гриб Анастасия',
  'Разместил пост/сторис в соц.сети с работой или результатом и отметил аккаунт школы @skillbox_Moskow',
  ' -',
  ' -',
  ' -',
  ' -'],
 ['Гриб Анастасия',
  'Участие в мероприятиях общих (просто участие желтая карточка)',
  ' -',
  ' -',
  ' -',
  ' -'],
 ['Гриб Анастасия',
  'Участие в метаворкшопе или спецворкшопе (зеленая карточка)',
  ' +',
  ' -',
  ' -',
  ' -'],
 ['Гриб Анастасия',
  'Участие в консультации (зеленая карточка)',
  ' +',
  ' -',
  ' -',
  ' -'],
 ['Гриб Анастасия', 'Пришел на онбординг', ' +', ' +', ' -', ' -'],
 ['Гриб Анастасия', '', ' -', ' -', ' -', ' -'],
 ['Гриб Анастасия', '', ' -', ' -', ' -', ' -'],
 ['Гриб Анастасия', '', ' -', ' -', ' -', ' -'],
 ['Гриб Анастасия', '', ' -', ' -', ' -', ' -'],
 ['Гриб Анастасия', '', ' -', ' -', ' -', ' -'],
 ['Гриб Анастасия', '', ' -', ' -', ' -', ' -'],
 ['Гриб Анастасия', '', ' -'

In [26]:
import pandas as pd

df = pd.DataFrame(result)
df = df[df[1] != '']

In [27]:
df[df[0] == "Гриб Анастасия"]

,0,1,2,3,4,5
0,Гриб Анастасия,Предложи эксперта для мероприятия в школе и со...,-,-,-,-
1,Гриб Анастасия,Разместил пост/сторис в соц.сети с работой или...,-,-,-,-
2,Гриб Анастасия,Участие в мероприятиях общих (просто участие ж...,-,-,-,-
3,Гриб Анастасия,Участие в метаворкшопе или спецворкшопе (зелен...,+,-,-,-
4,Гриб Анастасия,Участие в консультации (зеленая карточка),+,-,-,-
5,Гриб Анастасия,Пришел на онбординг,+,+,-,-
